In [1]:
import os
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
import scipy.misc
import pickle

%matplotlib inline

In [2]:
tf.enable_eager_execution()

In [ ]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
def resize_datasets(X):
    X_new = []
    for i in range(X.shape[0]):
        if(i % 100 == 0):
            print(i)
        img = scipy.ndimage.interpolation.zoom(X[i], (8.0, 8.0, 1.0))
        X_new.append(img)
    X_new = np.asarray(X_new, dtype=np.float32)
    return X_new.reshape([X.shape[0], 256, 256, 3])

In [ ]:
print('Resizing Validation...')
X_val = resize_datasets(X_val)

In [ ]:
print('Resizing Test...')
X_test = resize_datasets(X_test)

In [ ]:
print('Resizing Training...')
X_train = resize_datasets(X_train)

In [ ]:
X_train.shape

In [ ]:
pickle_out = open("train_dataset.pickle","wb")
pickle.dump(X_test, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("validation_dataset.pickle","rb")
X_val = pickle.load(pickle_in)

In [3]:
import pickle
pickle_in = open("dataset.pickle","rb")
X_train = pickle.load(pickle_in)
y_train = pickle.load(pickle_in)
X_test = pickle.load(pickle_in)
y_test = pickle.load(pickle_in)
X_val = pickle.load(pickle_in)
y_val = pickle.load(pickle_in)

In [4]:
X_test[:30] = X_train[:30]
y_test[:30] = y_train[:30]

In [5]:
training_set_size = 200
test_set_size = 100
valid_set_size = 50

In [6]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))

In [7]:
def flatten(x):
    """    
    Input:
    - TensorFlow Tensor of shape (N, D1, ..., DM)
    
    Output:
    - TensorFlow Tensor of shape (N, D1 * ... * DM)
    """
    N = tf.shape(x)[0]
    return tf.reshape(x, (N, -1))

In [ ]:
inp = tf.get_variable(name = 'input', shape=[1,256,256,3], dtype=tf.float32)

In [8]:
#globals used in the assignment
IMG_SHAPE = [256, 256, 3] # height, width, channels
NUM_CLASSES = 10

# training hyperparameters
LEARNING_RATE = 1e-4
MOMENTUM = 0.9
BATCH_SIZE = 16
EPOCHS = 30
DISPLAY_STEP = 1
VALIDATION_STEP = 1000
SAVE_STEP = 1000
CKPT_PATH = './ckpt'
SUMMARY_PATH = './summary'

# net architecture hyperparamaters
LAMBDA = 5e-4 #for weight decay
DROPOUT = 0.5

# test hyper parameters
K_PATCHES = 5
TOP_K = 5

In [9]:
def get_compatibility(atten1_output, g):
#     print("\nCompatibility ->")
#     print("Attention : " + str(atten1_output.numpy().shape)) #(batch, 225, 512)
#     print("G : " + str(g.numpy().shape)) #(batch, 512, 1)
    g_new = tf.reshape(g, (g.shape[0], g.shape[2], g.shape[1])) #(batch, 1, 512)
#     print("After reshaping, G : " + str(g_new.numpy().shape))
    g_new = tf.tile(g_new, [1, atten1_output.shape[1], 1]) #(batch, 225, 512)
    final_vec = (g_new + atten1_output)
#     print("After addition of G and L_i : " + str(final_vec.numpy().shape))
    return final_vec

In [20]:
class AlexNet(tfe.Network):
    def __init__(self, training):
        super(AlexNet, self).__init__()
        self.training = training
        
        # convolutional layers
        conv_init = tf.contrib.layers.xavier_initializer_conv2d()
        
        self.conv1 = self.track_layer(
            tf.layers.Conv2D(
                96,
                11,
                4,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))
        self.pool1 = self.track_layer(tf.layers.MaxPooling2D(3, 2, 'VALID'))

        self.conv2 = self.track_layer(
            tf.layers.Conv2D(
                256,
                5,
                1,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))
        self.pool2 = self.track_layer(tf.layers.MaxPooling2D(3, 2, 'VALID'))

        self.conv3 = self.track_layer(
            tf.layers.Conv2D(
                384,
                3,
                1,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))

        self.conv4 = self.track_layer(
            tf.layers.Conv2D(
                384,
                3,
                1,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))

        self.conv5 = self.track_layer(
            tf.layers.Conv2D(
                256,
                3,
                1,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))
        self.pool5 = self.track_layer(tf.layers.MaxPooling2D(3, 2, 'VALID'))

        # fully connected layers

        fc_init = tf.contrib.layers.xavier_initializer()

        self.fc1 = self.track_layer(
            tf.layers.Dense(
                512, kernel_initializer=fc_init))
        self.drop1 = self.track_layer(tf.layers.Dropout(DROPOUT))
        
        #perceptron
        self.perceptron1 = self.track_layer(
            tf.layers.Dense(
                1, kernel_initializer=fc_init))
        
        self.perceptron2 = self.track_layer(
            tf.layers.Dense(
                1, kernel_initializer=fc_init))
        
        #attention estimators
        self.atten1_fc = self.track_layer(
            tf.layers.Dense(
                512, kernel_initializer=fc_init))
        
        self.atten2_fc = self.track_layer(
            tf.layers.Dense(
                512, kernel_initializer=fc_init))
        
        self.out1 = self.track_layer(
            tf.layers.Dense(NUM_CLASSES, activation=tf.nn.softmax, kernel_initializer=fc_init))
        
        self.out2 = self.track_layer(
            tf.layers.Dense(NUM_CLASSES, activation=tf.nn.softmax, kernel_initializer=fc_init))

    def call(self, x):
        """ Function that executes the model """
        output = self.conv1(x)
        #print("After conv1 : " + str(output.numpy().shape))
        output = tf.nn.lrn(
            output, depth_radius=2, bias=1.0, alpha=2e-05, beta=0.75)
        #print("After normalisation1 : " + str(output.numpy().shape))
        output = self.pool1(output)
        #print("After pool1 : " + str(output.numpy().shape))

        output = self.conv2(output)
        #print("After conv2 : " + str(output.numpy().shape))
        output = tf.nn.lrn(
            output, depth_radius=2, bias=1.0, alpha=2e-05, beta=0.75)
        #print("After normalisation2 : " + str(output.numpy().shape))
        output = self.pool2(output)
        #print("After pool2 : " + str(output.numpy().shape))

        output = self.conv3(output)
        #print("After conv3 : " + str(output.numpy().shape))

        output = self.conv4(output)
        #print("After conv4 : " + str(output.numpy().shape))
        conv4_output = output

        output = self.conv5(output)
        #print("After conv5 : " + str(output.numpy().shape))
        conv5_output = output
        
        output = self.pool5(output)
        #print("After pool5 : " + str(output.numpy().shape))

        output = tf.layers.flatten(output)
        #print("After flattening : " + str(output.numpy().shape))

        output = self.fc1(output)
        #print("After fc1 : " + str(output.numpy().shape))
        if self.training:
            output = self.drop1(output)
            #print("After drop1 : " + str(output.numpy().shape))
        
#         logic for attention estimators
        g = tf.reshape(output, (output.shape[0], output.shape[1], 1))
#         print("G dimensions : " + str(g.numpy().shape))
        
#         print("\nATTENTION 1 :-")
        atten1_flattened_output = tf.reshape(conv4_output, (conv4_output.shape[0], conv4_output.shape[1] * conv4_output.shape[2], conv4_output.shape[3]))
#         print("After Flattening : " + str(atten1_flattened_output.numpy().shape))
        atten1_output = self.atten1_fc(atten1_flattened_output)
#         print("After FC : " + str(atten1_output.numpy().shape))
        atten1_output = self.perceptron1(get_compatibility(atten1_output, g))
#         print("After passing through perceptron : " + str(atten1_output.numpy().shape))
        atten1_output = tf.reshape(atten1_output, (atten1_output.shape[0], atten1_output.shape[2], atten1_output.shape[1]))
        attention1_values = atten1_output
        atten1_output = tf.nn.softmax(atten1_output)
#         print("After Reshaping and applying Softmax : " + str(atten1_output.numpy().shape))
        atten1_output = tf.matmul(atten1_output, atten1_flattened_output)
#         print("G_a1 Dimensions : " + str(atten1_output.numpy().shape))
        
#         print("\nATTENTION 2 :-")
        atten2_flattened_output = tf.reshape(conv5_output, (conv5_output.shape[0], conv5_output.shape[1] * conv5_output.shape[2], conv5_output.shape[3]))
#         print("After Flattening : " + str(atten2_flattened_output.numpy().shape))
        atten2_output = self.atten2_fc(atten2_flattened_output)
#         print("After FC : " + str(atten2_output.numpy().shape))
        atten2_output = self.perceptron2(get_compatibility(atten2_output, g))
#         print("After passing through perceptron : " + str(atten2_output.numpy().shape))
        atten2_output = tf.reshape(atten2_output, (atten2_output.shape[0], atten2_output.shape[2], atten2_output.shape[1]))
        attention2_values = atten2_output
        atten2_output = tf.nn.softmax(atten2_output)
#         print("After Reshaping and applying Softmax : " + str(atten2_output.numpy().shape))
        atten2_output = tf.matmul(atten2_output, atten2_flattened_output)
#         print("G_a2 Dimensions : " + str(atten2_output.numpy().shape))
        
#         print("\nCOMBINING BY OPTION 2 :-")
        output1 = tf.reshape(atten1_output, (atten1_output.shape[0], atten1_output.shape[2]))
#         print("Output1 Dimensions : " + str(output1.numpy().shape))
        output1 = self.out1(output1)
#         print("Output1 Dimensions : " + str(output1.numpy().shape))
        
        output2 = tf.reshape(atten2_output, (atten2_output.shape[0], atten2_output.shape[2]))
#         print("Output2 Dimensions : " + str(output2.numpy().shape))
        output2 = self.out2(output2)
#         print("Output2 Dimensions : " + str(output2.numpy().shape))
        
        output = tf.reduce_mean([output1, output2], 0)
        return output,attention1_values, attention2_values

In [21]:
def loss(net, mode, x, y):
    """
    Computes the loss for a given batch of examples
    Args:
        mode, string 'train' or 'val'
        x, tf tensor representing a batch of images
        y, tf tensor representing a batch of labels
    Returns:
        the loss between the predictions on the images and the groundtruths
    """
    pred = net(x)[0]
    scaled_y = tf.one_hot(y, NUM_CLASSES)

#     print(tf.log(pred).numpy().shape)
#     print(scaled_y.numpy().shape)
#     print(tf.reduce_sum(scaled_y * tf.log(pred), reduction_indices=[1]).numpy().shape)
    
    loss_value = tf.reduce_mean(-tf.reduce_sum(scaled_y * tf.log(pred), reduction_indices=[1]))
    
    weight_decay = tf.reduce_sum(LAMBDA * tf.stack([tf.nn.l2_loss(v) for v in net.variables]))

    total_loss = loss_value + weight_decay

    tf.contrib.summary.scalar(mode + '/loss', total_loss)

    return total_loss

In [22]:
def accuracy(net, mode, x, y):
    """
    Computes the accuracy for a given batch of examples
    Args:
        mode, string 'train' or 'val'
        x, tf tensor representing a batch of images
        y, tf tensor representing a batch of labels
    Returns:
        the accuracy of the predictions on the images and the groundtruths
    """
#     pred = tf.nn.softmax(net(x))
    pred = net(x)[0]
    y = tf.one_hot(y, NUM_CLASSES)

    accuracy_value = tf.reduce_sum(
                tf.cast(
                    tf.equal(
                        tf.argmax(pred, axis=1, output_type=tf.int64),
                        tf.argmax(y, axis=1, output_type=tf.int64)
                    ),
                    dtype=tf.float32
                ) 
            ) / float(pred.shape[0].value)

    tf.contrib.summary.scalar(mode +'/accuracy', accuracy_value)

    return accuracy_value

In [23]:
train_dset = Dataset(X_train, y_train, batch_size=BATCH_SIZE, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=valid_set_size, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=test_set_size)

In [24]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
#     print(y)
    if t > 5: break

0 (16, 256, 256, 3) (16,)
1 (16, 256, 256, 3) (16,)
2 (16, 256, 256, 3) (16,)
3 (16, 256, 256, 3) (16,)
4 (16, 256, 256, 3) (16,)
5 (16, 256, 256, 3) (16,)
6 (16, 256, 256, 3) (16,)


In [ ]:
img = X_val[1,:]
img = tf.to_float(img.reshape([1,256,256,3]))

In [ ]:
trainingset = Dataset(X_val[:8], y_val[:8], batch_size=BATCH_SIZE, shuffle=False)

In [25]:
#used for printing the log with the timestamp
logging = tf.logging
logging.set_verbosity(logging.INFO)
def log_msg(msg):
       logging.info(f'{time.ctime()}: {msg}') 

In [26]:
DISPLAY_STEP = 1
VALIDATION_STEP = 5
SAVE_STEP = 10

In [27]:
import time
def train(net):
    """
    Training procedure
    """
    start_time = time.time()
    step_time = 0.0
    
    writer = tf.contrib.summary.create_summary_file_writer(SUMMARY_PATH)
#     optimizer = tf.train.MomentumOptimizer(learning_rate=LEARNING_RATE, momentum=MOMENTUM)
    optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE)
    global_step = tf.train.get_or_create_global_step()
    epoch = tfe.Variable(0, name='epoch', dtype=tf.float32, trainable=False)
    all_variables = (net.variables + optimizer.variables() + [global_step] + [epoch])
    
    with writer.as_default():
        with tf.contrib.summary.record_summaries_every_n_global_steps(DISPLAY_STEP):
            for e in range(int(epoch.numpy()), EPOCHS):
                tf.assign(epoch, e)
                for (batch_i, (images, labels)) in enumerate(train_dset):
                    global_step = tf.train.get_global_step()
                    step = global_step.numpy() + 1
                    images = tf.to_float(images)
                    step_start_time = int(round(time.time() * 1000))
                    
                    optimizer.minimize(lambda: loss(net, 'train', images, labels), global_step=global_step)

                    step_end_time = int(round(time.time() * 1000))
                    step_time += step_end_time - step_start_time

                    if (step % DISPLAY_STEP) == 0:
                        l = loss(net, 'train', images, labels)
                        a = accuracy(net, 'train', images, labels).numpy()
                        log_msg('Epoch: {:03d} Step/Batch: {:09d} Step mean time: {:04d}ms \nLoss: {:.7f} Training accuracy: {:.4f}'.format(e, step, int(step_time / step), l, a))

                    if (step % VALIDATION_STEP) == 0:
                        for (_, (val_images, val_labels)) in enumerate(val_dset):
#                             val_images, val_labels = tfe.Iterator(val_dset).next()
                            l = loss(net, 'val', val_images, val_labels)
                            a = accuracy(net, 'val', val_images, val_labels).numpy()
                            int_time = time.time() - start_time
                            log_msg('Elapsed time: {:04d}ms --- Loss: {:.7f} Validation accuracy: {:.4f}'.format(int(int_time), l, a))

                    if (step % SAVE_STEP) == 0:
                        tfe.Saver(net.variables).save(os.path.join(CKPT_PATH, 'net.ckpt'), global_step=global_step)
                        log_msg('Variables saved')

In [28]:
alexnet = AlexNet(training=True)


Please inherit from `tf.keras.Model`, and see its documentation for details. `tf.keras.Model` should be a drop-in replacement for `tfe.Network` in most cases, but note that `track_layer` is no longer necessary or supported. Instead, `Layer` instances are tracked on attribute assignment (see the section of `tf.keras.Model`'s documentation on subclassing). Since the output of `track_layer` is often assigned to an attribute anyway, most code can be ported by simply removing the `track_layer` calls.

`tf.keras.Model` works with all TensorFlow `Layer` instances, including those from `tf.layers`, but switching to the `tf.keras.layers` versions along with the migration to `tf.keras.Model` is recommended, since it will preserve variable names. Feel free to import it with an alias to avoid excess typing :).


In [29]:
if not os.path.exists(CKPT_PATH):
    os.makedirs(CKPT_PATH)
train(alexnet)

INFO:tensorflow:Sat Nov 17 21:52:44 2018: Epoch: 000 Step/Batch: 000000002 Step mean time: 2063ms 
Loss: 2.9551308 Training accuracy: 0.1875
INFO:tensorflow:Sat Nov 17 21:52:50 2018: Epoch: 000 Step/Batch: 000000003 Step mean time: 2694ms 
Loss: 3.0393324 Training accuracy: 0.0625
INFO:tensorflow:Sat Nov 17 21:52:57 2018: Epoch: 000 Step/Batch: 000000004 Step mean time: 3093ms 
Loss: 3.0247860 Training accuracy: 0.1250
INFO:tensorflow:Sat Nov 17 21:53:04 2018: Epoch: 000 Step/Batch: 000000005 Step mean time: 3335ms 
Loss: 3.0160055 Training accuracy: 0.0000
INFO:tensorflow:Sat Nov 17 21:53:13 2018: Elapsed time: 0035ms --- Loss: 3.0160737 Validation accuracy: 0.1200
INFO:tensorflow:Sat Nov 17 21:53:19 2018: Epoch: 000 Step/Batch: 000000006 Step mean time: 3463ms 
Loss: 3.0048728 Training accuracy: 0.0625
INFO:tensorflow:Sat Nov 17 21:53:26 2018: Epoch: 000 Step/Batch: 000000007 Step mean time: 3554ms 
Loss: 3.0120447 Training accuracy: 0.0625
INFO:tensorflow:Sat Nov 17 21:53:33 2018: E

INFO:tensorflow:Sat Nov 17 21:59:02 2018: Elapsed time: 0384ms --- Loss: 2.6476531 Validation accuracy: 0.2200
INFO:tensorflow:Sat Nov 17 21:59:02 2018: Variables saved
INFO:tensorflow:Sat Nov 17 21:59:09 2018: Epoch: 003 Step/Batch: 000000051 Step mean time: 3827ms 
Loss: 2.5805106 Training accuracy: 0.1250
INFO:tensorflow:Sat Nov 17 21:59:15 2018: Epoch: 003 Step/Batch: 000000052 Step mean time: 3830ms 
Loss: 2.5867524 Training accuracy: 0.3750
INFO:tensorflow:Sat Nov 17 21:59:18 2018: Epoch: 003 Step/Batch: 000000053 Step mean time: 3797ms 
Loss: 2.3774862 Training accuracy: 0.2500
INFO:tensorflow:Sat Nov 17 21:59:25 2018: Epoch: 004 Step/Batch: 000000054 Step mean time: 3800ms 
Loss: 2.4705319 Training accuracy: 0.1250
INFO:tensorflow:Sat Nov 17 21:59:31 2018: Epoch: 004 Step/Batch: 000000055 Step mean time: 3807ms 
Loss: 2.8199401 Training accuracy: 0.0000
INFO:tensorflow:Sat Nov 17 21:59:39 2018: Elapsed time: 0421ms --- Loss: 2.6369467 Validation accuracy: 0.2000
INFO:tensorflow

INFO:tensorflow:Sat Nov 17 22:05:25 2018: Epoch: 007 Step/Batch: 000000100 Step mean time: 3869ms 
Loss: 2.1600614 Training accuracy: 0.4375
INFO:tensorflow:Sat Nov 17 22:05:33 2018: Elapsed time: 0775ms --- Loss: 2.4965856 Validation accuracy: 0.2600
INFO:tensorflow:Sat Nov 17 22:05:33 2018: Variables saved
INFO:tensorflow:Sat Nov 17 22:05:40 2018: Epoch: 007 Step/Batch: 000000101 Step mean time: 3870ms 
Loss: 2.3842523 Training accuracy: 0.3750
INFO:tensorflow:Sat Nov 17 22:05:46 2018: Epoch: 007 Step/Batch: 000000102 Step mean time: 3871ms 
Loss: 2.1906712 Training accuracy: 0.3125
INFO:tensorflow:Sat Nov 17 22:05:53 2018: Epoch: 007 Step/Batch: 000000103 Step mean time: 3873ms 
Loss: 2.3398173 Training accuracy: 0.3125
INFO:tensorflow:Sat Nov 17 22:05:59 2018: Epoch: 007 Step/Batch: 000000104 Step mean time: 3877ms 
Loss: 2.1698661 Training accuracy: 0.4375
INFO:tensorflow:Sat Nov 17 22:06:03 2018: Epoch: 007 Step/Batch: 000000105 Step mean time: 3859ms 
Loss: 1.9549043 Training ac

INFO:tensorflow:Sat Nov 17 22:11:52 2018: Epoch: 011 Step/Batch: 000000149 Step mean time: 3893ms 
Loss: 2.0007589 Training accuracy: 0.4375
INFO:tensorflow:Sat Nov 17 22:11:59 2018: Epoch: 011 Step/Batch: 000000150 Step mean time: 3894ms 
Loss: 2.0801888 Training accuracy: 0.3125
INFO:tensorflow:Sat Nov 17 22:12:07 2018: Elapsed time: 1169ms --- Loss: 2.6802835 Validation accuracy: 0.1400
INFO:tensorflow:Sat Nov 17 22:12:07 2018: Variables saved
INFO:tensorflow:Sat Nov 17 22:12:14 2018: Epoch: 011 Step/Batch: 000000151 Step mean time: 3895ms 
Loss: 1.8836741 Training accuracy: 0.5625
INFO:tensorflow:Sat Nov 17 22:12:20 2018: Epoch: 011 Step/Batch: 000000152 Step mean time: 3896ms 
Loss: 1.7333589 Training accuracy: 0.5000
INFO:tensorflow:Sat Nov 17 22:12:27 2018: Epoch: 011 Step/Batch: 000000153 Step mean time: 3898ms 
Loss: 2.0377066 Training accuracy: 0.4375
INFO:tensorflow:Sat Nov 17 22:12:33 2018: Epoch: 011 Step/Batch: 000000154 Step mean time: 3899ms 
Loss: 1.9391923 Training ac

INFO:tensorflow:Sat Nov 17 22:18:19 2018: Epoch: 015 Step/Batch: 000000198 Step mean time: 3904ms 
Loss: 2.0249414 Training accuracy: 0.4375
INFO:tensorflow:Sat Nov 17 22:18:26 2018: Epoch: 015 Step/Batch: 000000199 Step mean time: 3905ms 
Loss: 1.6832306 Training accuracy: 0.5625
INFO:tensorflow:Sat Nov 17 22:18:32 2018: Epoch: 015 Step/Batch: 000000200 Step mean time: 3905ms 
Loss: 1.7704493 Training accuracy: 0.4375
INFO:tensorflow:Sat Nov 17 22:18:40 2018: Elapsed time: 1562ms --- Loss: 2.6231530 Validation accuracy: 0.2200
INFO:tensorflow:Sat Nov 17 22:18:40 2018: Variables saved
INFO:tensorflow:Sat Nov 17 22:18:47 2018: Epoch: 015 Step/Batch: 000000201 Step mean time: 3907ms 
Loss: 1.6826578 Training accuracy: 0.6250
INFO:tensorflow:Sat Nov 17 22:18:54 2018: Epoch: 015 Step/Batch: 000000202 Step mean time: 3908ms 
Loss: 1.9938548 Training accuracy: 0.3750
INFO:tensorflow:Sat Nov 17 22:19:00 2018: Epoch: 015 Step/Batch: 000000203 Step mean time: 3908ms 
Loss: 1.8212581 Training ac

INFO:tensorflow:Sat Nov 17 22:25:04 2018: Epoch: 018 Step/Batch: 000000247 Step mean time: 3937ms 
Loss: 1.6251101 Training accuracy: 0.6250
INFO:tensorflow:Sat Nov 17 22:25:07 2018: Epoch: 018 Step/Batch: 000000248 Step mean time: 3930ms 
Loss: 1.2137653 Training accuracy: 0.8750
INFO:tensorflow:Sat Nov 17 22:25:14 2018: Epoch: 019 Step/Batch: 000000249 Step mean time: 3930ms 
Loss: 1.5080384 Training accuracy: 0.6250
INFO:tensorflow:Sat Nov 17 22:25:20 2018: Epoch: 019 Step/Batch: 000000250 Step mean time: 3931ms 
Loss: 1.6183541 Training accuracy: 0.6250
INFO:tensorflow:Sat Nov 17 22:25:28 2018: Elapsed time: 1970ms --- Loss: 2.6115565 Validation accuracy: 0.3000
INFO:tensorflow:Sat Nov 17 22:25:28 2018: Variables saved
INFO:tensorflow:Sat Nov 17 22:25:35 2018: Epoch: 019 Step/Batch: 000000251 Step mean time: 3932ms 
Loss: 1.3065500 Training accuracy: 0.6875
INFO:tensorflow:Sat Nov 17 22:25:42 2018: Epoch: 019 Step/Batch: 000000252 Step mean time: 3932ms 
Loss: 1.3615777 Training ac

INFO:tensorflow:Sat Nov 17 22:32:29 2018: Elapsed time: 2391ms --- Loss: 2.5615935 Validation accuracy: 0.2200
INFO:tensorflow:Sat Nov 17 22:32:40 2018: Epoch: 022 Step/Batch: 000000296 Step mean time: 4035ms 
Loss: 1.1681429 Training accuracy: 0.6250
INFO:tensorflow:Sat Nov 17 22:32:50 2018: Epoch: 022 Step/Batch: 000000297 Step mean time: 4042ms 
Loss: 1.3365033 Training accuracy: 0.7500
INFO:tensorflow:Sat Nov 17 22:33:01 2018: Epoch: 022 Step/Batch: 000000298 Step mean time: 4051ms 
Loss: 1.6313102 Training accuracy: 0.4375
INFO:tensorflow:Sat Nov 17 22:33:11 2018: Epoch: 022 Step/Batch: 000000299 Step mean time: 4059ms 
Loss: 1.3266174 Training accuracy: 0.6875
INFO:tensorflow:Sat Nov 17 22:33:17 2018: Epoch: 022 Step/Batch: 000000300 Step mean time: 4056ms 
Loss: 1.1133311 Training accuracy: 0.7500
INFO:tensorflow:Sat Nov 17 22:33:29 2018: Elapsed time: 2451ms --- Loss: 2.7434282 Validation accuracy: 0.2000
INFO:tensorflow:Sat Nov 17 22:33:30 2018: Variables saved
INFO:tensorflow

INFO:tensorflow:Sat Nov 17 22:40:09 2018: Epoch: 026 Step/Batch: 000000345 Step mean time: 4116ms 
Loss: 1.2023782 Training accuracy: 0.6875
INFO:tensorflow:Sat Nov 17 22:40:18 2018: Elapsed time: 2860ms --- Loss: 2.9568176 Validation accuracy: 0.2000
INFO:tensorflow:Sat Nov 17 22:40:26 2018: Epoch: 026 Step/Batch: 000000346 Step mean time: 4118ms 
Loss: 0.9175996 Training accuracy: 0.9375
INFO:tensorflow:Sat Nov 17 22:40:33 2018: Epoch: 026 Step/Batch: 000000347 Step mean time: 4119ms 
Loss: 0.9322172 Training accuracy: 0.9375
INFO:tensorflow:Sat Nov 17 22:40:41 2018: Epoch: 026 Step/Batch: 000000348 Step mean time: 4121ms 
Loss: 0.9805409 Training accuracy: 0.6875
INFO:tensorflow:Sat Nov 17 22:40:48 2018: Epoch: 026 Step/Batch: 000000349 Step mean time: 4123ms 
Loss: 1.0479108 Training accuracy: 0.7500
INFO:tensorflow:Sat Nov 17 22:40:56 2018: Epoch: 026 Step/Batch: 000000350 Step mean time: 4124ms 
Loss: 1.0585588 Training accuracy: 0.7500
INFO:tensorflow:Sat Nov 17 22:41:05 2018: E

In [30]:
for (_, (test_images, test_labels)) in enumerate(test_dset):
    l = loss(alexnet, 'val', test_images, test_labels)
    a = accuracy(alexnet, 'val', test_images, test_labels).numpy()
log_msg('Final Test Loss: {:.7f} Test accuracy: {:.4f}'.format(l, a))

INFO:tensorflow:Sat Nov 17 22:46:08 2018: Final Test Loss: 2.6794021 Test accuracy: 0.4500
